In [ ]:
from google.colab import files
files.upload()

In [ ]:
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv('train.csv', low_memory=False)
df.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,4.0,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",-1,NaN,11.27,4.0,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,_,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",5,4,6.27,4.0,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",6,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [ ]:
df.isna().sum()[df.isna().sum() > 0]

Name                        9985
Monthly_Inhand_Salary      15002
Type_of_Loan               11408
Num_of_Delayed_Payment      7002
Num_Credit_Inquiries        1965
Credit_History_Age          9030
Amount_invested_monthly     4479
Monthly_Balance             1200
dtype: int64

In [ ]:
df.drop(columns=['Name', 'SSN'], inplace=True)

In [ ]:

#Some Age values have an underscore and a minus, for the underscore we substitute it for "nothing" and change it's type to float.

df.Age = df.Age.str.replace('_', '', regex=True).astype('float')
df.Annual_Income = df.Annual_Income.str.replace('_', '', regex=True).astype('float')
df.Num_of_Loan = df.Num_of_Loan.str.replace('_', '', regex=True).astype('float')
df.Outstanding_Debt = df.Outstanding_Debt.str.replace('_', '', regex=True).astype('float')
df.Num_of_Delayed_Payment = df.Num_of_Delayed_Payment.str.replace('_', '', regex=True).astype('float')
df.Num_Bank_Accounts = df.Num_Bank_Accounts.astype('float')

In [ ]:
df.replace('_______', np.nan, inplace=True)
df.replace('_', np.nan, inplace=True)
df.replace('!@9#%8', np.nan, inplace=True)
df.replace('__10000__', np.nan, inplace=True)
df.Payment_of_Min_Amount.replace('NM', np.nan, inplace=True)
df.loc[df.Age > 100, 'Age'] = np.nan
df.loc[df.Age < 1, 'Age'] = np.nan
df.loc[df.Num_of_Loan < 0, 'Num_of_Loan'] = np.nan
df.loc[df.Num_of_Loan > 9, 'Num_of_Loan'] = np.nan
df.loc[df.Num_Credit_Card > 11, 'Num_Credit_Card'] = np.nan
df.loc[df.Interest_Rate > 34, 'Interest_Rate'] = np.nan
df.loc[df.Annual_Income > 300000, 'Annual_Income'] = np.nan
df.loc[0, 'Occupation'] = 'Scientist'
df.loc[df.Total_EMI_per_month > 5000, 'Total_EMI_per_month'] = np.nan
df.loc[df.Num_Bank_Accounts > 100, 'Num_Bank_Accounts'] = np.nan
df.loc[df.Num_Bank_Accounts < 0, 'Num_Bank_Accounts'] = np.nan
df.loc[df.Num_of_Delayed_Payment > 100, 'Num_of_Delayed_Payment'] = np.nan

In [ ]:
df.Age.describe()

count    97224.000000
mean        33.323274
std         10.778909
min         14.000000
25%         24.000000
50%         33.000000
75%         42.000000
max        100.000000
Name: Age, dtype: float64

In [ ]:
mean_columns = [
    'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Amount_invested_monthly', 'Monthly_Balance', 'Num_of_Loan',
    'Num_Credit_Card', 'Interest_Rate', 'Annual_Income', 'Monthly_Inhand_Salary', 'Total_EMI_per_month', 'Num_Bank_Accounts'
    ]
last_columns = ['Age', 'Occupation', 'Type_of_Loan', 'Credit_Mix']

In [ ]:
temp = df.groupby('Customer_ID')[last_columns].last()

In [ ]:
df.loc[df.Age.isnull(), 'Age'] = df.Customer_ID.map(temp.Age)
df.loc[df.Occupation.isnull(), 'Occupation'] = df.Customer_ID.map(temp.Occupation)
df.loc[df.Type_of_Loan.isnull(), 'Type_of_Loan'] = df.Customer_ID.map(temp.Type_of_Loan)
df.loc[df.Credit_Mix.isnull(), 'Credit_Mix'] = df.Customer_ID.map(temp.Credit_Mix)

In [ ]:
df.Credit_History_Age.fillna('0 0 0 0', inplace=True)

In [ ]:
for index, column in df.iterrows():
    df.loc[index, 'Credit_History_Age_Formated'] = float(df.loc[index, 'Credit_History_Age'].split()[0])*12+float(df.loc[index, 'Credit_History_Age'].split()[3])

In [ ]:
df.loc[df.Credit_History_Age_Formated == 0, 'Credit_History_Age_Formated'] = np.nan

In [ ]:
df.Credit_History_Age_Formated.interpolate(inplace = True)
df.drop(columns='Credit_History_Age', inplace=True)

In [ ]:
df.Payment_of_Min_Amount.fillna('No', inplace=True)

In [ ]:
df.Type_of_Loan.unique()

array(['Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan',
       'Credit-Builder Loan', 'Auto Loan, Auto Loan, and Not Specified',
       ..., 'Home Equity Loan, Auto Loan, Auto Loan, and Auto Loan',
       'Payday Loan, Student Loan, Mortgage Loan, and Not Specified',
       'Personal Loan, Auto Loan, Mortgage Loan, Student Loan, and Student Loan'],
      dtype=object)

In [ ]:
df.Type_of_Loan.fillna('Not Specified', inplace=True)

In [ ]:
df.drop(columns='Payment_Behaviour', inplace=True)

In [ ]:
digits = re.compile('[^\d.]')

In [ ]:
for index, column in df.iterrows():
    for x in mean_columns:
        df.loc[index, x] = digits.sub('', str(df.loc[index, x]))

In [ ]:
df.Num_of_Delayed_Payment.replace('', np.nan, inplace=True)
df.Num_of_Delayed_Payment = df.Num_of_Delayed_Payment.astype('float')
df.Changed_Credit_Limit.replace('', np.nan, inplace=True)
df.Changed_Credit_Limit = df.Changed_Credit_Limit.astype('float')
df.Num_Credit_Inquiries.replace('', np.nan, inplace=True)
df.Num_Credit_Inquiries = df.Num_Credit_Inquiries.astype('float')
df.Amount_invested_monthly.replace('', np.nan, inplace=True)
df.Amount_invested_monthly = df.Amount_invested_monthly.astype('float')
df.Monthly_Balance.replace('', np.nan, inplace=True)
df.Monthly_Balance = df.Monthly_Balance.astype('float')
df.Num_of_Loan.replace('', np.nan, inplace=True)
df.Num_of_Loan = df.Num_of_Loan.astype('float')
df.Num_Credit_Card.replace('', np.nan, inplace=True)
df.Num_Credit_Card = df.Num_Credit_Card.astype('float')
df.Interest_Rate.replace('', np.nan, inplace=True)
df.Interest_Rate = df.Interest_Rate.astype('float')
df.Annual_Income.replace('', np.nan, inplace=True)
df.Annual_Income = df.Annual_Income.astype('float')
df.Monthly_Inhand_Salary.replace('', np.nan, inplace=True)
df.Monthly_Inhand_Salary = df.Monthly_Inhand_Salary.astype('float')
df.Total_EMI_per_month.replace('', np.nan, inplace=True)
df.Total_EMI_per_month = df.Total_EMI_per_month.astype('float')
df.Num_Bank_Accounts.replace('', np.nan, inplace=True)
df.Num_Bank_Accounts = df.Num_Bank_Accounts.astype('float')

In [ ]:
df.Age.describe()

count    100000.000000
mean         33.322050
std          10.773792
min          14.000000
25%          24.000000
50%          33.000000
75%          42.000000
max         100.000000
Name: Age, dtype: float64

In [ ]:
mean_columns = [
    'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Amount_invested_monthly', 'Monthly_Balance', 'Num_of_Loan',
    'Num_Credit_Card', 'Interest_Rate', 'Annual_Income', 'Monthly_Inhand_Salary', 'Total_EMI_per_month', 'Num_Bank_Accounts'
    ]
last_columns = ['Age', 'Occupation', 'Type_of_Loan', 'Credit_Mix']

In [ ]:
temp = df.groupby('Customer_ID')[last_columns].last()

In [ ]:
df.loc[df.Age.isnull(), 'Age'] = df.Customer_ID.map(temp.Age)
df.loc[df.Occupation.isnull(), 'Occupation'] = df.Customer_ID.map(temp.Occupation)
df.loc[df.Type_of_Loan.isnull(), 'Type_of_Loan'] = df.Customer_ID.map(temp.Type_of_Loan)
df.loc[df.Credit_Mix.isnull(), 'Credit_Mix'] = df.Customer_ID.map(temp.Credit_Mix)

In [ ]:
print(df.columns)


Index(['ID', 'Customer_ID', 'Month', 'Age', 'Occupation', 'Annual_Income',
       'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card',
       'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan', 'Delay_from_due_date',
       'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Payment_of_Min_Amount',
       'Total_EMI_per_month', 'Amount_invested_monthly', 'Monthly_Balance',
       'Credit_Score', 'Credit_History_Age_Formated'],
      dtype='object')


In [ ]:
print(df.head())


       ID Customer_ID     Month   Age Occupation  Annual_Income  \
0  0x1602   CUS_0xd40   January  23.0  Scientist       19114.12   
1  0x1603   CUS_0xd40  February  23.0  Scientist       19114.12   
2  0x1604   CUS_0xd40     March  23.0  Scientist       19114.12   
3  0x1605   CUS_0xd40     April  23.0  Scientist       19114.12   
4  0x1606   CUS_0xd40       May  23.0  Scientist       19114.12   

   Monthly_Inhand_Salary  Num_Bank_Accounts  Num_Credit_Card  Interest_Rate  \
0            1824.843333                3.0              4.0            3.0   
1                    NaN                3.0              4.0            3.0   
2                    NaN                3.0              4.0            3.0   
3                    NaN                3.0              4.0            3.0   
4            1824.843333                3.0              4.0            3.0   

   Num_of_Loan                                       Type_of_Loan  \
0          4.0  Auto Loan, Credit-Builder Loan, Perso

In [ ]:
import pandas as pd

# Assuming 'Credit_History_Age' doesn't exist and you want to create it
df['Credit_History_Age'] = df.get('Credit_History_Age', '0 0 0 0')

# Now fill missing values in the newly created column
df['Credit_History_Age'].fillna('0 0 0 0', inplace=True)


In [ ]:
for index, column in df.iterrows():
    df.loc[index, 'Credit_History_Age_Formated'] = float(df.loc[index, 'Credit_History_Age'].split()[0])*12+float(df.loc[index, 'Credit_History_Age'].split()[3])

In [ ]:
df.loc[df.Credit_History_Age_Formated == 0, 'Credit_History_Age_Formated'] = np.nan

In [ ]:
df.Credit_History_Age_Formated.interpolate(inplace = True)
df.drop(columns='Credit_History_Age', inplace=True)

In [ ]:
df.Payment_of_Min_Amount.fillna('No', inplace=True)

In [ ]:
df.Type_of_Loan.unique()

array(['Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan',
       'Credit-Builder Loan', 'Auto Loan, Auto Loan, and Not Specified',
       ..., 'Home Equity Loan, Auto Loan, Auto Loan, and Auto Loan',
       'Payday Loan, Student Loan, Mortgage Loan, and Not Specified',
       'Personal Loan, Auto Loan, Mortgage Loan, Student Loan, and Student Loan'],
      dtype=object)

In [ ]:
df.Type_of_Loan.fillna('Not Specified', inplace=True)

In [ ]:
# Check if 'Payment_Behaviour' exists in the DataFrame
if 'Payment_Behaviour' in df.columns:
    # Drop the column if it exists
    df.drop(columns='Payment_Behaviour', inplace=True)
    print("Column 'Payment_Behaviour' dropped successfully.")
else:
    print("Column 'Payment_Behaviour' does not exist in the DataFrame.")


Column 'Payment_Behaviour' does not exist in the DataFrame.


In [ ]:
digits = re.compile('[^\d.]')

In [ ]:
for index, column in df.iterrows():
    for x in mean_columns:
        df.loc[index, x] = digits.sub('', str(df.loc[index, x]))

In [ ]:
df.Num_of_Delayed_Payment.replace('', np.nan, inplace=True)
df.Num_of_Delayed_Payment = df.Num_of_Delayed_Payment.astype('float')
df.Changed_Credit_Limit.replace('', np.nan, inplace=True)
df.Changed_Credit_Limit = df.Changed_Credit_Limit.astype('float')
df.Num_Credit_Inquiries.replace('', np.nan, inplace=True)
df.Num_Credit_Inquiries = df.Num_Credit_Inquiries.astype('float')
df.Amount_invested_monthly.replace('', np.nan, inplace=True)
df.Amount_invested_monthly = df.Amount_invested_monthly.astype('float')
df.Monthly_Balance.replace('', np.nan, inplace=True)
df.Monthly_Balance = df.Monthly_Balance.astype('float')
df.Num_of_Loan.replace('', np.nan, inplace=True)
df.Num_of_Loan = df.Num_of_Loan.astype('float')
df.Num_Credit_Card.replace('', np.nan, inplace=True)
df.Num_Credit_Card = df.Num_Credit_Card.astype('float')
df.Interest_Rate.replace('', np.nan, inplace=True)
df.Interest_Rate = df.Interest_Rate.astype('float')
df.Annual_Income.replace('', np.nan, inplace=True)
df.Annual_Income = df.Annual_Income.astype('float')
df.Monthly_Inhand_Salary.replace('', np.nan, inplace=True)
df.Monthly_Inhand_Salary = df.Monthly_Inhand_Salary.astype('float')
df.Total_EMI_per_month.replace('', np.nan, inplace=True)
df.Total_EMI_per_month = df.Total_EMI_per_month.astype('float')
df.Num_Bank_Accounts.replace('', np.nan, inplace=True)
df.Num_Bank_Accounts = df.Num_Bank_Accounts.astype('float')

In [ ]:
temp2 = df.groupby('Customer_ID')[mean_columns].mean()

In [ ]:
df.loc[df.Num_of_Delayed_Payment.isnull(), 'Num_of_Delayed_Payment'] = df.Customer_ID.map(temp2.Num_of_Delayed_Payment)
df.loc[df.Changed_Credit_Limit.isnull(), 'Changed_Credit_Limit'] = df.Customer_ID.map(temp2.Changed_Credit_Limit)
df.loc[df.Num_Credit_Inquiries.isnull(), 'Num_Credit_Inquiries'] = df.Customer_ID.map(temp2.Num_Credit_Inquiries)
df.loc[df.Amount_invested_monthly.isnull(), 'Amount_invested_monthly'] = df.Customer_ID.map(temp2.Amount_invested_monthly)
df.loc[df.Monthly_Balance.isnull(), 'Monthly_Balance'] = df.Customer_ID.map(temp2.Monthly_Balance)
df.loc[df.Num_of_Loan.isnull(), 'Num_of_Loan'] = df.Customer_ID.map(temp2.Num_of_Loan)
df.loc[df.Num_Credit_Card.isnull(), 'Num_Credit_Card'] = df.Customer_ID.map(temp2.Num_Credit_Card)
df.loc[df.Interest_Rate.isnull(), 'Interest_Rate'] = df.Customer_ID.map(temp2.Interest_Rate)
df.loc[df.Annual_Income.isnull(), 'Annual_Income'] = df.Customer_ID.map(temp2.Annual_Income)
df.loc[df.Total_EMI_per_month.isnull(), 'Total_EMI_per_month'] = df.Customer_ID.map(temp2.Total_EMI_per_month)
df.loc[df.Monthly_Inhand_Salary.isnull(), 'Monthly_Inhand_Salary'] = df.Customer_ID.map(temp2.Monthly_Inhand_Salary)
df.loc[df.Num_Bank_Accounts.isnull(), 'Num_Bank_Accounts'] = df.Customer_ID.map(temp2.Num_Bank_Accounts)

In [ ]:
df.head()

,ID,Customer_ID,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance,Credit_Score,Credit_History_Age_Formated
0,0x1602,CUS_0xd40,January,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7.0,11.27,4.0,Good,809.98,26.822620,No,49.574949,80.415295,312.494089,Good,NaN
1,0x1603,CUS_0xd40,February,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",-1,6.0,11.27,4.0,Good,809.98,31.944960,No,49.574949,118.280222,284.629162,Good,NaN
2,0x1604,CUS_0xd40,March,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7.0,10.27,4.0,Good,809.98,28.609352,No,49.574949,81.699521,331.209863,Good,NaN
3,0x1605,CUS_0xd40,April,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",5,4.0,6.27,4.0,Good,809.98,31.377862,No,49.574949,199.458074,223.451310,Good,NaN
4,0x1606,CUS_0xd40,May,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",6,6.0,11.27,4.0,Good,809.98,24.797347,No,49.574949,41.420153,341.489231,Good,NaN


In [ ]:
df.to_csv('train.csv', index=False)